# Airports around

By last, I will filter the remaining candidates by its proximity to any airport or airports. In order to accomplish that goal, I will use the Lufthansa Public API, which has a functionality that gives you the five closest airports of a given coordinate and its distance in kilometers. Let's perform the operation

In [1]:
from dotenv import load_dotenv
from bson import ObjectId
import json, requests
from statistics import mean
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
from folium.vector_layers import Circle, Polygon
import pandas as pd

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In [3]:
candidates = list(db.companies_usa_filt4_c.find({},{'coord':1}))

In [5]:
load_dotenv()
lufthansa_id = os.getenv('lufthansa_id')
lufthansa_secret = os.getenv('lufthansa_secret')

In [7]:
for cand in candidates:
    latitude = cand['coord']['coordinates'][1]
    longitude = cand['coord']['coordinates'][0]
    id = ObjectId(cand['_id'])
    
    request = 'https://api.lufthansa.com/v1/oauth/token'
    header_auth = {
                'client_id' : f'{lufthansa_id}',
                'client_secret': f'{lufthansa_secret}', 
                'grant_type' : 'client_credentials'}      

    r = requests.post(request, data = header_auth)
    j = r.json()
    token = j['access_token']
    header = { 'Authorization': 'Bearer ' + str(token), 'Accept': 'application/json'}

    resp = requests.get(f'https://api.lufthansa.com/v1/references/airports/nearest/{latitude},{longitude}?lang=en', headers=header)
    data_air = json.loads(resp.text)

    data_format = {}
    index = 0
    for airp in data_air['NearestAirportResource']['Airports']['Airport']:
        airp['Names']['Name']['name'] = airp['Names']['Name']['$']
        del airp['Names']['Name']['$']
        
        data_format[str(index)] = airp
        index += 1
    
    db.companies_usa_filt4_c.update_one({'_id':id}, {'$set':{'offices.airports': data_format}})

Now all our candidates have listed its five closest airports. Let's extract the median distance in kilometers for all of them, so we can rank the candidates.

In [8]:
airports = list(db.companies_usa_filt4_c.find({}, {'offices.airports':1}))

In [9]:
for airp in airports:
    id = ObjectId(airp['_id'])
    mean_dist = []
    for a in airp['offices']['airports']:
        mean_dist.append(airp['offices']['airports'][a]['Distance']['Value'])
    mean_add = mean(mean_dist)
    db.companies_usa_filt4_c.update_one({'_id':id}, {'$set':{'offices.airports.mean_dist': mean_add}})

In [10]:
dist_to_airp = list(db.companies_usa_filt4_c.find({}, {'offices.airports.mean_dist':1}))

In [11]:
unique_val = set([ap['offices']['airports']['mean_dist'] for ap in dist_to_airp])
unique_val

{42.2, 42.4, 42.6, 42.8, 43, 45.4, 60.6, 60.8, 61, 61.2, 86.6, 88.8}

There are mean values from 42 kilometers to 89. Let's take those which mean distance to its five airports is less than or equal to 42.6 kilometers.

In [16]:
offices_5 = list(db.companies_usa_filt4_c.find({'offices.airports.mean_dist':{'$lte':42.6}}))
[off['offices']['city'] for off in offices_5]

['New York',
 'Seattle',
 'New York',
 'New York',
 'Hoboken',
 'Hoboken',
 'Hoboken',
 'Hoboken',
 'New York',
 'New York',
 'New York',
 'Hoboken',
 'New York']

After apply the last filter I end up with 13 final candidates: seven of them in New York, and one in Seattle and five in Hoboken. Let's take them to a last collection and show the results in a map.

In [18]:
db.companies_usa_filt5_c.insert_many(offices_5)

# Map and conclussions

In [3]:
companies_df = pd.read_csv('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/geospatial-data-project/data/usa_docs/dataframes/companies_f5.csv')
schools_df = pd.read_csv('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/geospatial-data-project/data/usa_docs/dataframes/schools_f5.csv')
offices_df = pd.read_csv('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/geospatial-data-project/data/usa_docs/dataframes/offices_f5.csv')
airports_df = pd.read_csv('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/geospatial-data-project/data/usa_docs/dataframes/airports_f5.csv')

In [4]:
def drop_dup(dataframe):
    return dataframe.drop_duplicates()

offc_df = drop_dup(offices_df)
comp_df = drop_dup(companies_df)
school_df = drop_dup(schools_df)
airp_df = drop_dup(airports_df)

In [6]:
m_heat = Map(location=[40.737078, -74.034831],zoom_start=12)

fg_of = FeatureGroup(name='Offices')
for i, row in offc_df.iterrows():
    place = {"location":[row["latitude"],row["longitude"]], "popup":row["city"], "tooltip":row["state"]}
    ic = Icon(color='blue', prefix='glyphicon', icon='glyphicon-briefcase', icon_color='white')
    Marker(**place, icon=ic).add_to(fg_of)
fg_of.add_to(m_heat)

fg_sch = FeatureGroup(name='Schools')
for i, row in school_df.iterrows():
    place = {"location":[row["latitude"],row["longitude"]], "popup":row["name"], "tooltip":row["rating"]}
    ic = Icon(color='green', prefix='glyphicon', icon='glyphicon-globe', icon_color='black')
    Marker(**place, icon=ic).add_to(fg_sch)
fg_sch.add_to(m_heat)

fg_comp = FeatureGroup(name='Companies')
for i, row in comp_df.iterrows():
    place = {"location":[row["latitude"],row["longitude"]]}
    ic = Icon(color='red', prefix='glyphicon', icon='glyphicon-magnet', icon_color='black')
    Marker(**place, icon=ic).add_to(fg_comp)
fg_comp.add_to(m_heat)

fg_airp = FeatureGroup(name='Airports')
for i, row in airp_df.iterrows():
    place = {"location":[row["latitude"],row["longitude"]], 'popup':row['name'], 'tooltip':row['distance']}
    ic = Icon(color='orange', prefix='glyphicon', icon='glyphicon-plane', icon_color='black')
    Marker(**place, icon=ic).add_to(fg_airp)
fg_airp.add_to(m_heat)

LayerControl(collapsed=False).add_to(m_heat)
m_heat